# Libraries Used

**Importing all the relevant libraries that might be needed in the assignment:**

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import sqlite3
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


# Importing Data

Please Import the Assignment2023.sqlite file here to continue the code

In [ ]:
! wget https://github.com/PaulHancock/COMP5009_pracs/raw/main/data/Assignment2023.sqlite

In [ ]:
con = sqlite3.connect('Assignment2023.sqlite')
test_df = pd.read_sql_query('SELECT * FROM test;',con)
train_df = pd.read_sql_query('SELECT * FROM train;',con)

In [ ]:
# To take an overall look at the whole train data if done right
test_df


# Data Exploration

In [ ]:
# To take a look at the heads of both tables with the limit of 20
train_df.head(20)
test_df.head(20)

In [ ]:
# To check the shape of your data to see how many rows and columns it has
print("Test dataset shape:", test_df.shape)
print("Train dataset shape:", train_df.shape)


In [ ]:
# To Print the names of the columns
print("Column Names")
print(train_df.columns)

In [ ]:
# To describe each data attribute in more detail
train_df.describe()

# Data Preparation

**Dealing with Missing Values**

In [ ]:
# find which columns have missing data
def missing(df):

  missing_dict = dict()
  total = df.shape[0] # shape[0] is the number of rows
  for attribute in df.columns:
    missing = df[attribute].isna().sum() # count the number of Null/nan/na values
    frac = missing/total * 100 # as a percentage
    missing_dict[attribute] = frac
  return missing_dict

In [ ]:
m_dict = missing(train_df)
m_dict

In [ ]:
# To find out which attributes have missing values more than 50%
cols_to_drop = [ att for att,frac in m_dict.items() if frac >50]
cols_to_drop

In [ ]:
# As only one of them had more than 50 so we delete it from both train and test data set
cols_to_drop = ['Att24'] # Fill this in
train_df.drop(columns=cols_to_drop,
           inplace=True)
test_df.drop(columns=cols_to_drop,
           inplace=True)

In [ ]:
# confirm that our data frame now has fewer columns (was 280)
train_df.columns
test_df.columns

In [ ]:
# Now finding columns with missing values less than 50%
cols_to_impute = [ att for att,frac in m_dict.items() if 0<frac <50]
cols_to_impute

In [ ]:
train_df['Att07'].interpolate(method='linear', inplace=True)
test_df['Att07'].interpolate(method='linear', inplace=True)

train_df['Att15'].interpolate(method='linear', inplace=True)
test_df['Att15'].interpolate(method='linear', inplace=True)

In [ ]:
# find which columns have missing data
def missing(df):
  missing_dict = dict()
  total = df.shape[0] # shape[0] is the number of rows
  for attribute in df.columns:
    missing = df[attribute].isna().sum() # count the number of Null/nan/na values
    frac = missing/total * 100 # as a percentage
    missing_dict[attribute] = frac
  return missing_dict

In [ ]:
m_dict = missing(train_df)
m_dict

**Dealing with Duplicate Columns and Rows**

In [ ]:
# Finding duplicates in columns and rows for Train data

duplicate_columns = train_df.columns[train_df.columns.duplicated()]
if len(duplicate_columns) > 0:
    print("Duplicate columns found:")
    print(duplicate_columns)
else:
    print("No duplicate columns found.")


dups = train_df.duplicated()
dups.sum()

In [ ]:
dups = train_df.duplicated()
dups.sum()

In [ ]:
# Finding duplicates in columns and rows for Test data

duplicate_columns = test_df.columns[test_df.columns.duplicated()]
if len(duplicate_columns) > 0:
    print("Duplicate columns found:")
    print(duplicate_columns)
else:
    print("No duplicate columns found.")

dups = test_df.duplicated()
dups.sum()

**Data Types and Binarization**



In [ ]:
# To find the data types of each column
print("Data types")
print(train_df.dtypes)

In [ ]:
# To find out which columns are categorical
column_data_types = train_df.dtypes
categorical_columns = column_data_types[column_data_types == 'object'].index
print("Categorical Columns:", categorical_columns.tolist())

In [ ]:
# To find the the number of unique categories in the above mentioned columns
unique_categories_col1 = train_df['Att11'].nunique()
unique_categories_col2 = train_df['Att16'].nunique()
unique_categories_col3 = train_df['Att25'].nunique()

print("Unique categories in 'categorical_column1':", unique_categories_col1)
print("Unique categories in 'categorical_column2':", unique_categories_col2)
print("Unique categories in 'categorical_column3':", unique_categories_col3)


In [ ]:
# To find the frequency of occurance of each category in the dataset
categorical_columns = ['Att11', 'Att16', 'Att25']

for col_name in categorical_columns:
    unique_categories_counts = train_df[col_name].value_counts()

    print(f"Unique categories in '{col_name}':")
    for category, count in unique_categories_counts.iteritems():
        print(f"{category}: {count}")
    print()


In [ ]:
# Converting each into Numerical (Binarization) for train and test

new_Att11= pd.get_dummies(train_df.Att11,prefix="Att11")
new_Att16= pd.get_dummies(train_df.Att16,prefix="Att16")
new_Att25= pd.get_dummies(train_df.Att25,prefix="Att25")


new_Att11= pd.get_dummies(test_df.Att11,prefix="Att11")
new_Att16= pd.get_dummies(test_df.Att16,prefix="Att16")
new_Att25= pd.get_dummies(test_df.Att25,prefix="Att25")

In [ ]:
#merge Att11, Att16 and Att25 for training dataset
train_df_2 = pd.concat((train_df, new_Att11,new_Att16,new_Att25), axis=1)
test_df_2 = pd.concat((test_df, new_Att11,new_Att16,new_Att25), axis=1)

In [ ]:
# Dropping all the previous colunmns
train_df_2.drop(columns = 'Att11', inplace=True)
train_df_2.drop(columns = 'Att16', inplace=True)
train_df_2.drop(columns = 'Att25', inplace=True)

test_df_2.drop(columns = 'Att11', inplace=True)
test_df_2.drop(columns = 'Att16', inplace=True)
test_df_2.drop(columns = 'Att25', inplace=True)


In [ ]:
# Find the column index (position) of 'class'
class_column_index = train_df_2.columns.get_loc('class')

# Print the column index
print("Column Index of 'class':", class_column_index)


In [ ]:
#change order of the class to last column in train and test data
titles = list(train_df_2.columns)
titles[27],titles[-1] = titles[-1],titles[27]
train_df_2 = train_df_2[titles]

titles_test = list(test_df_2.columns)
titles_test[27],titles_test[-1] = titles_test[-1],titles_test[27]
test_df_2 = test_df_2[titles_test]

In [ ]:
print(train_df_2.columns)
train_df_2.head(40)

In [ ]:
#drop class attribute in the test_df_2 as that is what we need to predict by 0,1,2
test_df_2.drop(columns = 'class', inplace=True)

In [ ]:
 # It's necessary to store the "index" attribute as a variable for future arrangements.
ind = test_df['index']

**Finding Correlation between Attributes**

In [ ]:
cor = train_df_2.corr(method='pearson')
cor.style.background_gradient(cmap='coolwarm').set_precision(2)

In [ ]:
# Now deleting all attributes that are having correlation of more than 80%

train_df_2.drop(columns = 'Att00', inplace=True)
train_df_2.drop(columns = 'Att03', inplace=True)
train_df_2.drop(columns = 'Att06', inplace=True)
train_df_2.drop(columns = 'Att09', inplace=True)
train_df_2.drop(columns = 'Att13', inplace=True)
train_df_2.drop(columns = 'Att25_ASCZ', inplace=True)


test_df_2.drop(columns = 'Att00', inplace=True)
test_df_2.drop(columns = 'Att03', inplace=True)
test_df_2.drop(columns = 'Att06', inplace=True)
test_df_2.drop(columns = 'Att09', inplace=True)
test_df_2.drop(columns = 'Att13', inplace=True)
test_df_2.drop(columns = 'Att25_ASCZ', inplace=True)

In [ ]:
#KNn
X = train_df_2.iloc[:,:-1] #storing without the class attribute
y = train_df_2.iloc[:, -1] # y contains the class attribute

In [ ]:
X_test = test_df_2.iloc[:,:] #storing without class attribute in the test attribute

In [ ]:
y

In [ ]:
X

In [ ]:
X_test

**SCALING WITH Z SCORE**

In [ ]:
  # List of column names to convert to integer (replace with your actual column names)
columns_to_convert = ['Att11_AQDH', 'Att11_LCAS', 'Att11_NAAU', 'Att11_NBAE', 'Att11_QQNT',
                      'Att11_USGL', 'Att16_FBLE', 'Att16_ITRV', 'Att16_MFLQ', 'Att16_PWEH',
                      'Att16_QKNH', 'Att16_VVQP', 'Att16_XCYU', 'Att16_YCQC', 'Att16_YNCP',
                      'Att16_ZFBS', 'Att25_CDJW']

# Convert the selected columns to integer for train
X[columns_to_convert] = X[columns_to_convert].fillna(0).astype(int)
X[columns_to_convert] = X[columns_to_convert].fillna(0).astype(int)


In [ ]:
# List of column names to convert to integer (replace with your actual column names)
columns_to_convert = ['Att11_AQDH', 'Att11_LCAS', 'Att11_NAAU', 'Att11_NBAE', 'Att11_QQNT',
                      'Att11_USGL', 'Att16_FBLE', 'Att16_ITRV', 'Att16_MFLQ', 'Att16_PWEH',
                      'Att16_QKNH', 'Att16_VVQP', 'Att16_XCYU', 'Att16_YCQC', 'Att16_YNCP',
                      'Att16_ZFBS', 'Att25_CDJW']

# Convert the selected columns to integer for test data
X_test[columns_to_convert] = X_test[columns_to_convert].fillna(0).astype(int)
X_test[columns_to_convert] = X_test[columns_to_convert].fillna(0).astype(int)


In [ ]:
# To check if they are in integer now
print("Data types")
print(X.dtypes)

In [ ]:
# Initialize an empty list to store float64 column names
float_columns = []

# Iterate through the DataFrame's columns
for col in X.columns:
    if X[col].dtype == 'float64':
        float_columns.append(col)


In [ ]:
scaler = StandardScaler()
X_training = scaler.fit_transform(X[float_columns])
X_testing = scaler.fit_transform(X_test[float_columns])

In [ ]:
#splitting the dataset into two subsets
# train data is 75% and test data is 25%
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_training, y,
                                                    test_size=0.25,
                                                    random_state=4)

In [ ]:
#total 5000 elements divided 75% into train data and 25% into test data
X_train.shape, X_test.shape

In [ ]:
X.describe()

# Data classification

**Classification Model**

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn import tree
from sklearn.model_selection import StratifiedKFold, KFold, ShuffleSplit
from sklearn.metrics import accuracy_score
from imblearn.over_sampling import SMOTE
from collections import Counter
from sklearn import naive_bayes
from sklearn.ensemble import RandomForestClassifier

In [ ]:
skf = StratifiedKFold(n_splits=10)
kf = KFold(n_splits=10)
ss = ShuffleSplit(n_splits=10, test_size=15, random_state=4)

**MODEL IMBALANCE CHECK**

In [ ]:
train_df_2.groupby(['class']).count()

In [ ]:
#plot to show unbalanced nature of the 'class' attribute in train_df_2
LABELS = ['2', '1', '0']
count_classes = pd.value_counts(train_df_2['class'], sort=True)
colors = ['lightcoral', 'lightskyblue', 'lightgreen']
count_classes.plot(kind='pie', rot=0, autopct='%1.1f%%', colors=colors)
plt.title('Distribution of "Class" attribute')
plt.xlabel('Class')
plt.ylabel('Frequency')

plt.show()

In [ ]:
#SMOTE needs to be applied to balance the 'class' attribute and then check it by doing KNN
smote = SMOTE()

X_train_smote, y_train_smote = smote.fit_resample(X_train.astype("float"), y_train)
print("Before SMOTE: ", Counter(y_train))
print("After SMOTE: ", Counter(y_train_smote))

**KNN METHOD**

In [ ]:
# Create a dictionary of all the parameters we'll be iterating over
parameters = {'weights': ['uniform','distance'],
              'n_neighbors':[1,3,7,11]} #  list of the nearest neigbhours
# make a classifier object
knn = KNeighborsClassifier()
# create a GridSearchCV object to do the training with cross validation
gscv = GridSearchCV(estimator=knn,
                    param_grid=parameters,
                    cv=skf,  # the cross validation folding pattern
                    scoring='accuracy')
# now train our model
best_knn = gscv.fit(X_train, y_train)
best_knn.best_params_, best_knn.best_score_

In [ ]:
knn = KNeighborsClassifier(n_neighbors=11,weights='distance')
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)

In [ ]:
#train the model
best_knn.fit(X_train, y_train)
y_predict = best_knn.predict(X_test)
print("accuracy:", accuracy_score(y_test,y_predict)*(100),'%')
pd.crosstab(y_test,y_predict)

In [ ]:
from sklearn.metrics._plot.confusion_matrix import confusion_matrix
cfm = confusion_matrix(y_test, y_predict)
cfm

In [ ]:
from sklearn import metrics
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

In [ ]:
# KNN after balancing the attribute
best_knn.fit(X_train_smote, y_train_smote)
y_predict = best_knn.predict(X_test)
print("accuracy:", accuracy_score(y_test,y_predict)*(100),'%')
pd.crosstab(y_test,y_predict)

**Decision Tree Classifier**

In [ ]:
parameters = {'criterion': ('gini','entropy'),  # this should be the different splitting criteria
              'min_samples_split':[3,13,15,20]} # this should be the different values for min_samples_split
dtc = tree.DecisionTreeClassifier()
gscv = GridSearchCV(estimator=dtc,
                    param_grid=parameters,
                    cv=ss,
                    scoring='accuracy')
best_dtc = gscv.fit(X_train, y_train)
best_dtc.best_params_, best_dtc.best_score_

In [ ]:
X_train_dtc,y_train_dtc = X_train,y_train
dtc = tree.DecisionTreeClassifier(criterion = 'entropy',min_samples_split= 13)
dtc.fit(X_train_dtc, y_train_dtc)
y_pred = dtc.predict(X_test)

In [ ]:
print("Accuracy:",accuracy_score(y_test, y_pred))

In [ ]:
#apply decision tree model after balancing the elements
best_dtc.fit(X_train_smote, y_train_smote)
y_predict = best_dtc.predict(X_test)
print("accuracy:", accuracy_score(y_test,y_predict)*(100),'%')
pd.crosstab(y_test,y_predict)

**Naive Bayes Classifier**

In [ ]:
# no parameters to adjust so no need to optimise, just train
fig, ax = plt.subplots(1,1)
# Training our decision tree model
nb = naive_bayes.GaussianNB()
nb.fit(X_train, y_train)
y_predict = nb.predict(X_test)
print("accuracy:", accuracy_score(y_test,y_predict)*(100),'%')

class_labels = ["0.0", "1.0", "2.0"]
ConfusionMatrixDisplay.from_estimator(nb,
                                      X_test, y_test,
                                      display_labels=class_labels,  # Provide your class labels here
                                      ax=ax)

plt.tight_layout()
plt.show()


In [ ]:
#KN-Neighbor model with the balanced 'class' attribute in test dataset
best_knn.fit(X_train_smote, y_train_smote)
y_predict_knn = best_knn.predict(X_testing)

In [ ]:
# Decision tree classifier model with the balanced 'class' attribute in test dataset
best_dtc.fit(X_train_smote, y_train_smote)
y_predict_dtc = best_dtc.predict(X_testing)

In [ ]:
dataset_knn = pd.DataFrame(y_predict_knn)
dataset_dtc = pd.DataFrame(y_predict_dtc)

In [ ]:
# Concatenate our predictions with the index column
prediction = pd.concat((ind,dataset_knn,dataset_dtc), axis=1)
prediction.columns =['index', 'Predict1', 'Predict2']



In [ ]:
prediction['Predict1'] = prediction['Predict1'].astype(int)
prediction['Predict2'] = prediction['Predict2'].astype(int)

#display the results to check the format
prediction

In [ ]:
#save results in the SQL file
save_location = '/content/Answers.sqlite'
con = sqlite3.connect('Answers.sqlite')
prediction.to_sql(name='Answers.sqlite', con=con)
con.close()